# TSMAI sheet02 2025

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.linear_model import LinearRegression
import scipy.signal as sg
import scipy.io

## Task 1.1

Auto Regressive time series of order p: 

$$\text{AR}(p) : x_t = a_0 + \sum_{i=1}^p a_i x_{t-i} + \epsilon_t $$


$$
\mathbf{v} = \begin{pmatrix}
v_1 \\
v_2 \\
\vdots \\
v_n
\end{pmatrix}
$$

$$
\mathbf{A} = \begin{pmatrix}
a_{11} & a_{12} & \cdots & a_{1n} \\
a_{21} & a_{22} & \cdots & a_{2n} \\
\vdots & \vdots & \ddots & \vdots \\
a_{m1} & a_{m2} & \cdots & a_{mn}
\end{pmatrix}
$$

## Task 2

### Task 2.1

In [ ]:
data = scipy.io.loadmat('ex2file1.mat')
print(data.keys())

dict_keys(['__header__', '__version__', '__globals__', 'DLPFC1', 'DLPFC2', 'Parietal1', 'Parietal2'])


In [17]:
dlpfc1 = data['DLPFC1'].flatten()
dlpfc2 = data['DLPFC2'].flatten()
parietal1 = data['Parietal1'].flatten()
parietal2 = data['Parietal2'].flatten()

X = np.column_stack([dlpfc1, dlpfc2, parietal1, parietal2])

var_data = pd.DataFrame(X, columns=['DLPFC1', 'DLPFC2', 'Parietal1', 'Parietal2'])
print(var_data)
var_data = var_data.to_numpy()
print(var_data)

      DLPFC1   DLPFC2  Parietal1  Parietal2
0    2.28550 -0.29516   -3.03620   -4.51840
1    0.85457 -2.63380   -4.52230   -1.90250
2   -0.26151 -3.08110   -4.71460    0.47887
3   -1.19410 -2.62330   -3.97540    1.92820
4   -1.27900 -1.61420   -2.01040    3.77230
..       ...      ...        ...        ...
355 -2.13940 -2.24390   -0.60644   -4.15580
356 -1.32430 -2.30090   -1.01920   -3.81420
357 -1.62290 -3.47130   -2.59440   -4.29070
358 -1.31880 -3.15880   -2.61050   -3.56520
359 -0.51186 -2.72170   -2.49090   -1.62220

[360 rows x 4 columns]
[[ 2.2855  -0.29516 -3.0362  -4.5184 ]
 [ 0.85457 -2.6338  -4.5223  -1.9025 ]
 [-0.26151 -3.0811  -4.7146   0.47887]
 ...
 [-1.6229  -3.4713  -2.5944  -4.2907 ]
 [-1.3188  -3.1588  -2.6105  -3.5652 ]
 [-0.51186 -2.7217  -2.4909  -1.6222 ]]


In [20]:
class regVAR:
    def __init__(self, data):
        self.data = data

    def regress(self):
        X = self.data[:-1]
        y = self.data[1:]
        model = LinearRegression(fit_intercept=True).fit(X,y)
        A = model.coef_.T  # Transpose to match VAR convention
        intercept = model.intercept_

        return A, intercept


In [21]:
test = regVAR(var_data)
A, intercept = test.regress()

print('coeff matrix A : ')
print(A)

print('a0 vector : ')
print(intercept)

coeff matrix A : 
[[ 0.90668653  0.00975233 -0.01223154  0.15398133]
 [-0.02035257  0.89853182  0.04526295 -0.0979815 ]
 [ 0.01568962 -0.00518382  0.87266454  0.01444375]
 [-0.03375742 -0.00724422  0.00158463  0.93634082]]
a0 vector : 
[ 0.0024904  -0.00690006  0.00287008  0.03277537]


Var(1) is stationary Iff all eigenvalues of the coeffiecient matrix $|\lambda| < 1$.

In [38]:
max(abs(np.linalg.eigvals(A))) < 1

True

Checking the max eigenvalue and confirming that it is less then 1, and thus the VAR model is stationary.

In [24]:
class VAR_model:
    def __init__(self, coeff: list, init: np.ndarray, noise: str = 'normal', **kwargs):
        # 1. COEFFICIENTS: From scalars to matrices/vectors -------------------------
        # coeff = [intercept_vector, A1 (lag-1 matrix), A2 (lag-2 matrix), ...]
        self.order = len(coeff) - 1
        self.coeff = [np.array(c) for c in coeff]  # Convert all to arrays
        self.n_vars = self.coeff[1].shape[0]       # Number of variables (e.g., 4)

        # 2. INITIALIZATION: Now a 2D array (lagged values for all variables) -------
        # init shape = (order, n_vars), e.g., (2, 4) for VAR(2) with 4 regions
        self.init = np.array(init)
        self.xn_temp = self.init.copy()  # Stores full time series history
        self.xn = None

        # 3. NOISE: Now multivariate (e.g., multivariate normal) --------------------
        self.noise = noise
        self.config = kwargs  # Should include 'mean' (vector) and 'cov' (matrix)


    def __call__(self, length: int):
        # 4. OUTPUT: 2D array (time × variables) ------------------------------------
        xt_arr = np.zeros((length, self.n_vars))

        if self.noise == 'normal':
            mean = self.config.get('mean', np.zeros(self.n_vars))
            cov = self.config.get('cov', np.eye(self.n_vars))

            for i in range(length):
                # 5. LAGGED TERMS: Matrix multiplications instead of scalars -------
                lagged_terms = np.zeros(self.n_vars)
                for lag in range(1, self.order + 1):
                    A_lag = self.coeff[lag]  # Coefficient matrix for this lag
                    lagged_vector = self.xn_temp[-lag, :]  # Get lagged values
                    lagged_terms += A_lag @ lagged_vector  # Matrix-vector multiply

                # 6. INTERCEPT: Now a vector instead of scalar ----------------------
                noise = np.random.multivariate_normal(mean, cov)
                xt = self.coeff[0] + lagged_terms + noise

                # Update history
                xt_arr[i, :] = xt
                self.xn_temp = np.vstack((self.xn_temp, xt))

            # Append to full history
            if self.xn is not None:
                self.xn = np.vstack((self.xn, xt_arr))
            else:
                self.xn = xt_arr

            return xt_arr